In [ ]:
#고객 유지를위한 행동을 예측하십시오. 
#모든 관련 고객 데이터를 분석하고 집중된 고객 유지 프로그램을 개발할 수 있습니다.

#고객이 구독을 취소했는지 여부를 지정하는 이탈 레이블과 함께 정리 된 고객 활동 데이터 (기능)로 구성된 
# Orange Telecom의 Churn Dataset은 예측 모델을 개발하는 데 사용됩니다. 
#여기에서 두 가지 데이터 세트를 사용할 수 있습니다. churn-80 및 churn-20 데이터 세트를 다운로드 할 수 있습니다.

#두 세트는 동일한 배치에서 나왔지만 80/20 비율로 분할되었습니다. 
# ML 모델을 개발하는 데 더 많은 데이터가 필요한 경우가 많으므로 학습 및 교차 검증 목적으로 더 큰 세트 
#(즉, churn-80)를 사용하고 최종 테스트 및 모델 성능을 위해 더 작은 세트 (즉, churn-20)를 사용하겠습니다.

In [ ]:
#churn80은 train, churn20은 test용

# 전처리

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

churn_train=pd.read_csv("C:/Users/SAMSUNG/Desktop/train_churn.csv")
churn_test=pd.read_csv("C:/Users/SAMSUNG/Desktop/test_churn.csv")
print(churn_train.shape)
churn_train.head(3)

In [ ]:
churn_test.info()

In [ ]:
print(churn_train['Churn'].value_counts())
print(churn_test['Churn'].value_counts())

In [ ]:
churn_train.describe()

In [ ]:
import pandas as pd

churn_train=pd.get_dummies(churn_train[['State','International plan','Voice mail plan']]) # 원핫인코딩 실행
churn_test=pd.get_dummies(churn_test[['State','International plan','Voice mail plan']]) # 원핫인코딩 실행

In [ ]:
churn_train.info()

In [ ]:
X_train = churn_train.iloc[:,:-1]
y_train = churn_train.iloc[:,-1]
X_test = churn_test.iloc[:,:-1]
y_test = churn_test.iloc[:,-1]

# 모델적합_XGBoost

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

xgb_clf = XGBClassifier(n_estimators=500, random_state=0,booster='gbtree')

xgb_clf.fit(X_train,y_train,early_stopping_rounds=100,
           eval_metric="auc", eval_set=[(X_train,y_train),(X_test,y_test)])

xgb_roc_score = roc_auc_score(y_test, xgb_clf.predict_proba(X_test)[:,1],average='macro')

In [ ]:
from sklearn.model_selection import GridSearchCV

xgb_clf = XGBClassifier(n_estimators=100)

params = {'max_depth':[5,7],'min_child_weight':[1,3],'colsample_bytree':[0.5,0.75]}

gridcv=GridSearchCV(xgb_clf,param_grid=params)
gridcv.fit(X_train,y_train,early_stopping_rounds=30,eval_metric="auc",
          eval_set=[(X_train,y_train),(X_test,y_test)])

print('GridSearchCV optimum paramater:', gridcv.best_params_)

xgb_roc_score = roc_auc_score(y_test,gridcv.predict_proba(X_test)[:,1],average='macro')
print('ROC AUC: {0:.4f}'.format(xgb_roc_score))

In [ ]:
#GridSearchCV optimum paramater: {'colsample_bytree': 0.5, 'max_depth': 5, 'min_child_weight': 1}
#ROC AUC: 1.0000

In [ ]:
from xgboost import plot_importance
from matplotlib import pyplot

xgb_clf = XGBClassifier(n_estimators=100,random_state=0,colsample_bytree=0.5,max_depth=5,min_child_weight=1)
xgb_clf.fit(X_train,y_train,early_stopping_rounds=200
            ,eval_metric="auc",eval_set=[(X_train,y_train),(X_test,y_test)])

In [ ]:
# plot feature importance
plot_importance(xgb_clf)
pyplot.show()

#과적합의 문제
#변수중요도에서 국가 변수에 중요성이 치중되어 있음

# 모델 적합_LightGBM

In [ ]:
from lightgbm import LGBMClassifier

lgbm_clf = LGBMClassifier(n_estimators=500)

evals = [(X_test,y_test)]
lgbm_clf.fit(X_train,y_train,early_stopping_rounds=100,eval_metric="auc",eval_set=evals,
            verbose=True)

lgbm_roc_score=roc_auc_score(y_test,lgbm_clf.predict_proba(X_test)[:,1],average='macro')
print('ROC AUC: {0:.4f}'.format(lgbm_roc_score))

In [ ]:
from lightgbm import plot_importance
import matplotlib.pyplot as plt

plot_importance(lgbm_clf)
pyplot.show()

#과적합, 변수중요도 판단불가;;